In [ ]:
ls -lh *_1.fq.gz | cut  -d " " -f10 > read_name_1.txt
ls -lh *_2.fq.gz | cut  -d " " -f10 > read_name_2.txt
grep "map04020" shibanyu.Gmo.expan.gene.difkegg.filt.xls | awk -F "\t" '{print$10}'|sed "s/ /\n/g"|for i in $(cat -);do grep $i shibanyu.Gmo.expan.gene.anno.xls;done > result.xls

In [ ]:
trimmomatic PE -threads 20 -phred33 C_PL_L1a_1.fq.gz C_PL_L1a_2.fq.gz ./trimmomatic/C_PL_L1a_1P.fq.gz ./trimmomatic/C_PL_L1a_1U.fq.gz ./trimmomatic/C_PL_L1a_2P.fq.gz ./trimmomatic/C_PL_L1a_2U.fq.gz ILLUMINACLIP:/home/guisen/biosoftware/miniconda/share/trimmomatic/adapters/TruSeq3-PE.fa:2:30:10 LEADING:5 TRAILING:5 SLIDINGWINDOW:5:20  MINLEN:50

#mapping
hisat2-build -p 22 ~/genome-Ple/genome.fa Ple-genome
hisat2 -t -p 20 -x ~/thesis/02mapping/index__hisat2/Ple-genome --dta -1 ~/thesis/00all_read/trimmomatic/C_PL_L1b_1P.fq.gz -2 ~/thesis/00all_read/trimmomatic/C_PL_L1b_2P.fq.gz -S C_PL_L1b.sam
samtools sort C_PL_L1a.sam  -o C_PL_L1a.bam

#transcription
conda install -c bioconda stringtie
stringtie C_PL_L1a.bam -p 20 -o C_PL_L1a.gtf
for i in $(ls ./samfile/|sed "s/.sam//g");do echo "nohup samtools sort ./samfile/$i.sam -o $i.bam &" | bash ;done
ls ./*.gtf|sed "s/ /\n/" > gtffile.list
stringtie --merge -p22 -o merge.gtf gtffile.list
gtf_genome_to_cdna_fasta.pl merge.gtf ~/genome-Ple/genome.fa > merge.fasta
gtf_to_alignment_gff3.pl merge.gtf > merge.gff3
TransDecoder.LongOrfs -t merge.fasta 
TransDecoder.Predict -t merge.fasta
cdna_alignment_orf_to_genome_orf.pl merge.fasta.transdecoder.gff3  merge.gff3 merge.fasta > transcripts.gff3

#de nove
augustus --species=help
augustus --species=zebrafish --gff3=on --start=off --stop=off --stopCodonExcludedFromCDS=false ~/genome-Ple/genome.fa--outfile=augustus.gff3

#homology
tblastn -query refprots.fa -db target_genome.fa -outfmt 6  -num_threads 22 -max_target_seqs 10 -evalue 1e-5 > tblastn_output.tab

#EVidenceModeler

~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/gff3_gene_prediction_file_validator.pl gene.gff #验证gff可用
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/partition_EVM_inputs.pl --genome genome.fasta --gene_predictions gene.gff --segmentSize 100000 --overlapSize 10000 --partition_listing partitions_list.out > partition.log 2>&1 &
~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/write_EVM_commands.pl --genome genome.fasta --weights /home/guisen/weights.txt --gene_predictions gene.gff --output_file_name evm.out  --partitions partitions_list.out >  commands.list
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/execute_EVM_commands.pl commands.list >> ./execule.log 2>&1 &
~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/recombine_EVM_partial_outputs.pl  --partitions partitions_list.out --output_file_name evm.out
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/convert_EVM_outputs_to_GFF3.pl --partitions partitions_list.out --output evm.out --genome ~/genome-Ple/genome.fa &
find . -regex ".*evm.out.gff3" -exec cat {} \; > EVM.all.gff3

gffread EVM.all.gff -g input/genome.fa -y tr_cds.fa
bioawk -c fastx '$seq < 50 {print $comment}' tr_cds.fa | cut -d '=' -f 2 > short_aa_gene_list.txt
grep -v -w -f short_aa_gene_list.txt EvM.all.gff > filter.gff

"""
de nove : ABINITIO_PREDICTION      AUGUSTUS  3
homology : OTHER_PREDICTION      genBlastG       7
transcription : OTHER_PREDICTION        transdecoder    10
"""

nohup /home/guisen/biosoftware/miniconda/share/transposonPSI/transposonPSI.pl cds.fa  nuc > denove.log 2>&1 &

In [ ]:
conda activate guisen
export PERL5LIB=/public/home/xuyue/anaconda3/envs/guisen/bin/perl5.26.2usr/lib/ >> ~/.bashrc

bsub -n 5 -o partition.log ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/partition_EVM_inputs.pl --genome ~/guisen/genome_file/Ple-genome.fa --gene_predictions all.gff --segmentSize 100000 --overlapSize 10000 --partition_listing partitions_list.out
bsub -n 5 -o commands.list  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/write_EVM_commands.pl --genome ../genome_file/Ple-genome.fa  --weights /public/home/xuyue/guisen/evidencemodeler/weights.txt --gene_predictions all.gff --output_file_name evm.out  --partitions partitions_list.out
bsub -n 5 -o execule.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/execute_EVM_commands.pl commands.list
bsub -n 5 -o recombine.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/recombine_EVM_partial_outputs.pl  --partitions partitions_list.out --output_file_name evm.out
bsub -n 5 -o convert.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/convert_EVM_outputs_to_GFF3.pl --partitions partitions_list.out --output evm.out --genome ../genome_file/Ple-genome.fa
find . -regex ".*evm.out.gff3" -exec cat {} \; > EVM.all.gff3
bioawk -c fastx 'length($seq) < 300 {print $name}' cds.fa > short_aa_gene_list.txt
grep -v -w -f short_aa_gene_list.txt EvM.all.gff > filter.gff
bsub -n 5 -o run.log /public/home/xuyue/anaconda3/share/transposonPSI/transposonPSI.pl cds.fa nuc

#busco
conda install -c bioconda -c conda-forge busco=5.0.0
busco -i ~/pep.fa -o busco -l actinopterygii_odb10  -m prot -c 20
generate_plot.py -wd ./

#拆分nr数据库
conda install -c bioconda taxonkit
conda install -c bioconda csvtk 
wget -b -c https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz
wget -b -c https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
taxonkit list -j 20 --ids 2759 --indent "" --data-dir ./ > Eukaryota_list
zcat prot.accession2taxid.gz |csvtk -t grep -f taxid -P Eukaryota_list |csvtk  -t cut -f accession.version > Eukaryotacc.txt
blastdbcmd -db nr_Eukaryota -entry all -dbtype prot -out nr_Eukaryota.fa
makeblastdb -in nr_Eukaryota.fa -dbtype prot -title nr_Eukaryota -parse_seqids -out nr_Eukaryota
blastp -db ~/database/blast/taxid/nr_Eukaryota  -query test.fa -evalue 1e-5 -num_threads 20 -max_target_seqs 1 -out blast.xls -outfmt 6
bsub -n 60 -q fat -o run.log blastp -db ~/guisen/database/nr_Eukaryota/nr_Eukaryota  -query ~/guisen/blast_result/pep.Ple.fa  -evalue 1e-5 -num_threads 60 -max_target_seqs 1 -out blast.xls -outfmt 6
sed -i 's/>//;s/ >/\n/g;s/\s/\t/' nr_Eukaryota.xls

#爬取kegg数据库
wget -c -b http://rest.kegg.jp/list/ko
cat ../all.fish.genes.list|cut -f1|while read id; do echo "nohup wget -c http://rest.kegg.jp/get/$id/aaseq -O $id  >> nohup.log 2>&1 & "|bash; done
bsub -n 60 -q fat -o run.log blastp -db kegg-fish/kegg_fish -query ~/guisen/blast_result/pep.Ple.fa  -evalue 1e-5 -num_threads 20 -max_target_seqs 1 -out kegg.xls -outfmt 6

#interproscan注释
nohup interproscan.sh -i ../pep.Ple.fa  -iprlookup -goterms  -dp -T temp  -f tsv -cpu 20 -o interpro.xls &

#expression
for i in $(cat ./sample.list);do bsub -c 10 -o Ple.log stringtie -e -B -p 10 -G ~/guisen/gff_file/Ple.gff3 -o ./Ple/${i}/${i}.gtf ./${i}.bam;done

#readcount
bsub -q fat -c 30 -o run.log samtools sort -n ../../C_PL_L1a.bam -o C_PL_L1a.nsort.bam
htseq-count -f bam -r name -s no -a 10 -t gene -i ID -m union sort_sam_name/C_PL_L1a.nsort.bam  ~/guisen/gff_file/Ple.gff3 > C_PL_L1a.count.xls

#DESeq2
library(DESeq2)
setwd('G:/thesis/05gene_Expression_Quantification/HTseq_reads_count')
count_reads = read.csv('reads_count_matrix_filt.xls',sep='\t',header=T)
row.names(count_reads) = count_reads$gene_id
coldata= read.csv('sample.xls',sep='\t',header=T)
dds = DESeqDataSetFromMatrix(countData = count_reads[2:25],colData = coldata,design = ~group)
dds = DESeq(dds) #标准化
res = results(dds,contrast = c('group','PL_La','C_PL_La'))
#res = results(dds,contrast = c('group','PL_Lb','C_PL_Lb'))
#res = results(dds,contrast = c('group','PL_Sa','C_PL_Sa'))
#res = results(dds,contrast = c('group','PL_Sb','C_PL_Sb'))
summary(res)
diff_gene_deseq2 = subset(res,padj < 0.05 & (log2FoldChange > 2 | log2FoldChange < -2)) #差异基因筛选条件
resdata =  merge(as.data.frame(res),as.data.frame(counts(dds,normalize=TRUE))[1:6],by="row.names",sort=FALSE)
#resdata = merge(as.data.frame(res),as.data.frame(counts(dds,normalize=TRUE))[7:12],by="row.names",sort=FALSE)
#resdata = merge(as.data.frame(res),as.data.frame(counts(dds,normalize=TRUE))[13:18],by="row.names",sort=FALSE)
#resdata = merge(as.data.frame(res),as.data.frame(counts(dds,normalize=TRUE))[19:24],by="row.names",sort=FALSE)
write.cvs
#valcano
"""
ggplot(resdata,aes(x =log2FoldChange, y=-log10(padj),color=change))
+ geom_point(size=1.5,na.rm = T)
+ labs(title = 'DESeq2') # 添加Plot.title
+ theme_grey(base_size=8) # 设置预设主题
+ theme(text = element_text(family='serif',size=10,face = 'bold')) # 设置所有字体为Times New Roman,10号，粗体
+ geom_vline(xintercept=c(-2, 2), lty=2, col="gray", lwd=0.5) # 设置垂直线，线型虚线，颜色灰色，线宽0.5
+ geom_hline(yintercept=-log10(0.05), lty=2, col="gray", lwd=0.5) # 设置水平线
+ scale_color_manual(name="significant", values=c("#f8766d", "#00ba38", "#619cff"),breaks =c("Up", "Down", "Unchanged"),labels = paste(c('Down','Unchanged','Up'),table(resdata$change),sep=':')[c(3,1,2)]) #设置图列
"""
resdata$change <- as.factor(ifelse(resdata$padj<0.05 & abs(resdata$log2FoldChange)>2,ifelse(resdata$log2FoldChange>1, "Up", "Down"),"Unchanged"))
ggplot(resdata,aes(x =log2FoldChange, y=-log10(padj), color=change))+ geom_point(size=1.5)+ theme_grey(base_size=8)+ labs(title = 'DESeq2',hjust=0.5)+ theme(text = element_text(family='serif',size=10))+geom_vline(xintercept=c(-2, 2), lty=2, col="gray", lwd=0.5)+geom_hline(yintercept=-log10(0.05), lty=2, col="gray", lwd=0.5)+scale_color_manual(name="", values=c("#f8766d", "#00ba38", "#619cff"), limits=c("Up", "Down", "Unchanged"))+guides(color=guide_legend(title='significant'))
"""


In [ ]:
#Eak-TLR
fastq-dump --gzip --split-3 --defline-qual '+' --defline-seq '@\$ac-\$si/\$ri' SRX49538*
for i in $(seq -w 66 78);do echo "bsub -n 10 -o run${i}.log trimmomatic PE -threads 10 -phred33 SRX49538${i}_1.fastq.gz  SRX49538${i}_2.fastq.gz ./trimmomatic/SRX49538${i}_1P.fastq.gz ./trimmomatic/SRX49538${i}_1U.fastq.gz ./trimmomatic/SRX49538${i}_2P.fastq.gz ./trimmomatic/SRX49538${i}_2U.fastq.gz ILLUMINACLIP:/public/home/xuyue/anaconda3/share/trimmomatic/adapters/TruSeq3-PE.fa:2:30:10 LEADING:5 TRAILING:5 SLIDINGWINDOW:5:20  MINLEN:50"|bash;done
for i in $(seq -w 66 78);do echo "bsub -n 10 -o maping${i}.log hisat2 -t -p 10 -x Eak-genome --dta -1 ./sra/trimmomatic/SRX49538${i}_1P.fastq.gz -2 ./sra/trimmomatic/SRX49538${i}_2P.fastq.gz -S SRX49538${i}.sam"|bash;done
for i in $(seq -w 66 78);do echo "bsub -n 10 -o samtools${i}.log samtools sort SRX49538${i}.sam -o SRX49538${i}.bam"|bash ;done
for i in $(seq -w 66 78);do echo "bsub -n 10 -o run.log stringtie -e -B -p 10 -G ./rsg.gene.gff -o ./stringtie/SRX49538${i}/SRX49538${i}.gtf SRX49538${i}.bam"|bash;done
for i in $(seq -w 66 78);do echo "SRX49538${i} ./SRX49538${i}/SRX49538${i}.gtf";done > sample.txt ; sed -i 's/\s/\t/' sample.txt
prepDE.py -i sample.txt

library(ballgown)
bg=ballgown(dataDir='G:/Eak-TLRs/express/stringtie',samplePattern='SRX',meas='all')
transcript_fpkm=texpr(bg,'all')
library(pheatmap)
tlr = read.csv('G:/Eak-TLRs/express/TLR.csv',header = T,sep=',',row.names = 1)
pheatmap(tlr[c(1:8,10)],scale = 'row',cluster_rows = F,color = colorRampPalette(c("navy", "white", "firebrick3"))(50),display_numbers = F,cellwidth = 20, cellheight = 18)

In [ ]:
coldata <- data.frame(row.names=colnames(data)[2:25],group = factor(rep(c('C_PL_La','PL_La','C_PL_Lb','PL_Lb','C_PL_Sa','PL_Sa','C_PL_Sb','PL_Sb'), each = 3)))
dds <- DESeqDataSetFromMatrix(countData = data[2:25], colData = coldata, design= ~group)
rld = rlog(dds)
plotPCA(rlog,intgroup=c('group'))